In [1]:
import xarray as xr
from ocf_blosc2 import Blosc2
from tqdm import tqdm
%load_ext autoreload
%autoreload 2

In [2]:
import json
with open("indices.json") as f:
    site_locations = {
        data_source: {
            int(site): (int(location[0]), int(location[1]))
            for site, location in locations.items()
        }
        for data_source, locations in json.load(f).items()
    }

In [3]:
from dataset import ChallengeDataset

In [4]:
ds = ChallengeDataset('/data/pv/2021/', '/data/satellite-nonhrv_proc/2021/', "/data/weather_proc/2021/", site_locations, None)

In [ ]:
import sys
item = ds[0]
ctx = 0
for i in item:
    ctx += i.nbytes
print(ctx * len(ds))

In [ ]:
list(ds.open_parquet('/data/pv/2021/1.parquet')) == list(ds.open_parquet('/data/pv/2021/1.parquet').xs("2021-01-01 08:00:00").index)

In [ ]:
len(ds.open_parquet('/data/pv/2021/1.parquet').index)

In [ ]:
for i in range(100):
    pv_features, hrv_features, weather_features, extra, pv_targets = ds[i]
    print(hrv_features.shape, weather_features.shape)

In [ ]:
del results

In [ ]:
import h5py
import numpy as np
from datetime import datetime
with h5py.File("/data/weather_proc/2021/1.hdf5", "r") as hdf_file:
    time = np.array(hdf_file['time'])
    time = np.array(list(map(lambda x: datetime.utcfromtimestamp(int(x)), time)))
    n = np.where((time == datetime(2021, 1, 1, 8, 0)))[0][0]
    nth_values = []
    SKIP_LIST = set(['latitude', 'longitude', 'time'])
    for name, dataset in hdf_file.items():
        # Check if the item is a dataset
        if name in SKIP_LIST: 
            continue
        print(name)
        if isinstance(dataset, h5py.Dataset):
            # Fetch the nth value from the dataset
            # Ensuring n is within the bounds of the dataset's size
            if n < len(dataset):
                nth_value = dataset[n]
                # Append the nth value as a numpy array
                nth_values.append(np.array(nth_value))
            else:
                # Handle cases where n exceeds dataset size
                print(f"Dataset '{name}' does not have a {n}th value.")

In [ ]:
x = 100
y = 100
nth_values[0][y - 64 : y + 64, x - 64 : x + 64].shape